In [54]:
import os
os.chdir("../")
%pwd

'd:\\1Acads_local\\AI_related_projects\\github_projects\\text_summeriser_project\\Text_summerizer_website_intiail'

In [55]:
import os
import numpy as np
import pandas as pd

In [56]:
from dataclasses import dataclass
from pathlib import Path

## Ingeneral when we create a class we need to wrap that with th decorator dataclass(frozen = True) -->to automatically define __init__,__repr__,__eq__ ,to make the class immutable(frozen = True) That is any instance of this class once declraed can't be changed latter like a constant variable
@dataclass(frozen=True)
class DataingestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [57]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.textSummarizer.constants import config_yaml_file_path, prams_yaml_file_path
from src.textSummarizer.utils.common import read_yaml, create_directories

In [62]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = config_yaml_file_path,
        params_filepath = prams_yaml_file_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion_config

        create_directories([config.root_dir])

        data_ingestion = DataingestionConfig(
            root_dir=config.root_dir,
            source_url =config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion

In [63]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging.logging import logger
from src.textSummarizer.utils.common import get_size

In [78]:
import os
import requests
import zipfile
import logging
from pathlib import Path

class DataIngestion:
    def __init__(self, config):
        self.config = config
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def download_file(self):
        try:
            source_url = self.config.source_url
            local_file = self.config.local_data_file

            # Create directory if needed
            os.makedirs(os.path.dirname(local_file), exist_ok=True)

            # Download with proper headers
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }

            response = requests.get(source_url, headers=headers, stream=True)
            response.raise_for_status()

            # Write file in chunks
            with open(local_file, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            # Verify downloaded file
            if not os.path.exists(local_file):
                raise Exception("Download failed - file not created")
            
            if not zipfile.is_zipfile(local_file):
                os.remove(local_file)
                raise Exception("Downloaded file is not a valid ZIP")

            self.logger.info(f"Download successful: {local_file}")
            return local_file

        except Exception as e:
            self.logger.error(f"Download failed: {str(e)}")
            raise

    def extract_zip_file(self):
        try:
            local_file = self.config.local_data_file
            unzip_path = self.config.unzip_dir
            
            if not os.path.exists(local_file):
                raise FileNotFoundError(f"ZIP file not found: {local_file}")

            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(local_file, 'r') as zip_ref:
                # Test ZIP file integrity before extraction
                if zip_ref.testzip() is not None:
                    raise zipfile.BadZipFile("ZIP file is corrupted")
                zip_ref.extractall(unzip_path)

            self.logger.info(f"Extraction successful: {unzip_path}")

        except Exception as e:
            self.logger.error(f"Extraction failed: {str(e)}")
            raise

In [79]:
from src.textSummarizer.constants import * 
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-23 12:26:09,654: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-23 12:26:09,659: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-23 12:26:09,661: INFO: common: created directory at: artifacts]
[2024-12-23 12:26:09,662: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-23 12:27:08,499: INFO: 1128719105: Download successful: artifacts/data_ingestion/data.zip]
[2024-12-23 12:27:08,741: INFO: 1128719105: Extraction successful: artifacts/data_ingestion]


In [74]:
import requests
import io
import zipfile
import binascii

def verify_zip_file(url):
    """
    Verify if URL points to a valid ZIP file by checking signatures
    """
    try:
        # Setup headers
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        # Download first 4 bytes to check signature
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        
        # Read first 4 bytes
        header = next(response.iter_content(4))
        
        # ZIP file signature is 50 4B 03 04 in hex
        zip_signature = b'PK\x03\x04'
        
        if header == zip_signature:
            print(f"Valid ZIP signature detected: {binascii.hexlify(header).decode()}")
            return True
        else:
            print(f"Invalid ZIP signature: {binascii.hexlify(header).decode()}")
            return False
            
    except Exception as e:
        print(f"Error checking ZIP file: {str(e)}")
        return False

# Test the URL
url = "https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip"
if verify_zip_file(url):
    print("File appears to be a valid ZIP")
else:
    print("File is not a valid ZIP")

Valid ZIP signature detected: 504b0304
File appears to be a valid ZIP
